Negative set: whole genome negatives vs shuffled reference 

Single tasking vs multi-tasking for SPI1, CTCF, ZNF143, SIX5 

Wish list: Multi-input model w/ DNAse + Sequence as input  (Abhi,outperforms factor net within same cell type) 

## Input data 

We will learn to predict transcription factor binding for four transcription factors in the GM12878 cell line (one of the Tier 1 cell lines for the ENCODE project). First, we download the narrowPeak bed files for each of these transcription factors. You can skip the following code block if you already have the data downloaded. 

In [ ]:
## CTCF, optimal IDR thresholded peaks, Stam Lab, hg19
# https://www.encodeproject.org/experiments/ENCSR000DRZ/
!wget https://www.encodeproject.org/files/ENCFF473RXY/@@download/ENCFF473RXY.bed.gz 

## SPI1, optimal IDR thresholded peaks, Myers lab, hg19
# https://www.encodeproject.org/experiments/ENCSR000BGQ/
!wget https://www.encodeproject.org/files/ENCFF002CHQ/@@download/ENCFF002CHQ.bed.gz
    
## ZNF143, optimal IDR thresholded peaks, Snyder lab, hg19
#https://www.encodeproject.org/experiments/ENCSR936XTK/
!wget https://www.encodeproject.org/files/ENCFF544NXC/@@download/ENCFF544NXC.bed.gz

## SIX5, optimal IDR thresholded peaks, Myers Lab, hg19
# https://www.encodeproject.org/experiments/ENCSR000BJE/
!wget https://www.encodeproject.org/files/ENCFF606WUV/@@download/ENCFF606WUV.bed.gz


## Case 1: Negatives consist of shuffled references, single-tasked models

## Case 2: Whole-genome negatives, single-tasked models 

## Case 3: Negatives consist of shuffled references, multi-tasked models 

## Case 4: Whole-genome negatives, multi-tasked models 